# Pandas

**joining data frames**

In [1]:
import pandas as pd

`pd.concat`

In [9]:
df1 = pd.DataFrame(
    {
        'id': [1, 2, 3],
        'name': ['Alice', 'Bob', 'John']
    }
)
df2 = pd.DataFrame(
    {
        'id': [4, 5, 6, 7],
        'name': ['Jane', 'Adam', 'Josh', 'Another']
    }
)

In [10]:
df1

,id,name
0,1,Alice
1,2,Bob
2,3,John


In [11]:
df2

,id,name
0,4,Jane
1,5,Adam
2,6,Josh
3,7,Another


In [12]:
pd.concat([df1, df2])

,id,name
0,1,Alice
1,2,Bob
2,3,John
0,4,Jane
1,5,Adam
2,6,Josh
3,7,Another


In [13]:
pd.concat([df1, df2], axis=1)

,id,name,id,name
0,1.0,Alice,4,Jane
1,2.0,Bob,5,Adam
2,3.0,John,6,Josh
3,NaN,NaN,7,Another


`pd.merge`

In [27]:
df1 = pd.DataFrame(
    {
        'id': [1, 2, 3],
        'name': ['Alice', 'Bob', 'John']
    }
)
df2 = pd.DataFrame(
    {
        'id': [2, 3, 4],
        'name': ['Bob1', 'John', 'Unknown'],
        'score': [50, 60, 70]
    }
)
df1

,id,name
0,1,Alice
1,2,Bob
2,3,John


In [28]:
df2

,id,name,score
0,2,Bob1,50
1,3,John,60
2,4,Unknown,70


**default inner join**

In [32]:
result = pd.merge(df1, df2, on='id', suffixes=[None, '_other'])
result

,id,name,name_other,score
0,2,Bob,Bob1,50
1,3,John,John,60


In [34]:
result = pd.merge(df1, df2, on='id')
result.columns = ['id', 'name', 'name_other', 'score']
result

,id,name,name_other,score
0,2,Bob,Bob1,50
1,3,John,John,60


In [38]:
df1 = pd.DataFrame(
    {
        'id': [1, 2, 3],
        'name': ['Alice', 'Bob', 'John']
    }
)
df2 = pd.DataFrame(
    {
        'id': [2, 3, 4],
        'score': [50, 60, 70]
    }
)
df1

,id,name
0,1,Alice
1,2,Bob
2,3,John


In [40]:
df2

,id,score
0,2,50
1,3,60
2,4,70


In [41]:
result = pd.merge(df1, df2, on='id', how='left')
result

,id,name,score
0,1,Alice,NaN
1,2,Bob,50.0
2,3,John,60.0


In [42]:
result = pd.merge(df1, df2, on='id', how='right')
result

,id,name,score
0,2,Bob,50
1,3,John,60
2,4,NaN,70


In [43]:
result = pd.merge(df1, df2, on='id', how='outer')
result

,id,name,score
0,1,Alice,NaN
1,2,Bob,50.0
2,3,John,60.0
3,4,NaN,70.0


In [45]:
result.sort_values(by=['score'])

,id,name,score
1,2,Bob,50.0
2,3,John,60.0
3,4,NaN,70.0
0,1,Alice,NaN


In [46]:
df1

,id,name
0,1,Alice
1,2,Bob
2,3,John


In [47]:
df2

,id,score
0,2,50
1,3,60
2,4,70


In [50]:
res_temp = pd.merge(df1, df2, on='id', how='left')
res_temp

,id,name,score
0,1,Alice,NaN
1,2,Bob,50.0
2,3,John,60.0


In [53]:
res_temp[res_temp['score'].isna()].drop(columns=['score'])

,id,name
0,1,Alice


**grouping**

In [89]:
data = {
    'Department': ['HR', 'HR', 'IT', 'IT', 'Finance', 'Finance', 'Finance'],
    'Employee': ['Alice', 'Bob', 'Charlie', 'David', 'Eva', 'Frank', 'John'],
    'Salary': [50000, 60000, 70000, 80000, 90000, 100000, 110000]
}
df = pd.DataFrame(data)
df

,Department,Employee,Salary
0,HR,Alice,50000
1,HR,Bob,60000
2,IT,Charlie,70000
3,IT,David,80000
4,Finance,Eva,90000
5,Finance,Frank,100000
6,Finance,John,110000


In [57]:
df['Salary'].max()

np.int64(100000)

In [59]:
unique_departments = df['Department'].drop_duplicates().tolist()
unique_departments

['HR', 'IT', 'Finance']

In [65]:
for department in unique_departments:
    filtered = df[df['Department'] == department]
    # display(filtered)
    # print('-'*50)
    max_sal = filtered['Salary'].max()
    print(f'Max salary for the {department!r} is {max_sal}')

Max salary for the 'HR' is 60000
Max salary for the 'IT' is 80000
Max salary for the 'Finance' is 100000


In [69]:
result = df.groupby('Department')['Salary']
type(result)

pandas.core.groupby.generic.SeriesGroupBy

In [70]:
result

In [71]:
result.max()

Department
Finance    100000
HR          60000
IT          80000
Name: Salary, dtype: int64

In [72]:
result.min()

Department
Finance    90000
HR         50000
IT         70000
Name: Salary, dtype: int64

In [75]:
max_df = df.groupby('Department')['Salary'].max()
min_df = df.groupby('Department')['Salary'].min()
pd.merge(max_df, min_df, left_index=True, right_index=True)

,Salary_x,Salary_y
Department,,
Finance,100000,90000
HR,60000,50000
IT,80000,70000


In [84]:
max_df.reset_index(drop=True)

0    100000
1     60000
2     80000
Name: Salary, dtype: int64

In [77]:
max_df.index

Index(['Finance', 'HR', 'IT'], dtype='object', name='Department')

In [86]:
df.groupby('Department')['Salary'].agg(['min', 'max', 'mean']).reset_index()

,Department,min,max,mean
0,Finance,90000,100000,95000.0
1,HR,50000,60000,55000.0
2,IT,70000,80000,75000.0


In [96]:
import numpy as np

def salary_range(x):
    return x.max() - x.min()

df.groupby('Department')['Salary'].agg(['max', 'min', 'mean', salary_range])

,max,min,mean,salary_range
Department,,,,
Finance,110000,90000,100000.0,20000
HR,60000,50000,55000.0,10000
IT,80000,70000,75000.0,10000


In [120]:
data = {
    'Department': ['HR', 'HR', 'IT', 'IT', 'Finance', 'Finance'],
    'Employee': ['Alice', 'Bob', 'Charlie', 'David', 'Eva', 'Frank'],
    'Salary': [50000, 60000, 70000, 80000, 90000, 100000],
    'Bonus': [5000, 6000, 7000, 8000, 9000, 10000]
}
df = pd.DataFrame(data)
df

,Department,Employee,Salary,Bonus
0,HR,Alice,50000,5000
1,HR,Bob,60000,6000
2,IT,Charlie,70000,7000
3,IT,David,80000,8000
4,Finance,Eva,90000,9000
5,Finance,Frank,100000,10000


In [103]:
df.groupby('Department')[['Salary', 'Bonus']].agg(['max','min'])

Salary         Bonus      
               max    min    max   min
Department                            
Finance     100000  90000  10000  9000
HR           60000  50000   6000  5000
IT           80000  70000   8000  7000

In [121]:
result = df.groupby('Department').agg({
    'Salary': ['max', 'min', 'mean'],
    'Bonus': ['sum', 'mean']
})
result

Salary                  Bonus        
               max    min     mean    sum    mean
Department                                       
Finance     100000  90000  95000.0  19000  9500.0
HR           60000  50000  55000.0  11000  5500.0
IT           80000  70000  75000.0  15000  7500.0

In [122]:
result.columns.get_level_values(0)

Index(['Salary', 'Salary', 'Salary', 'Bonus', 'Bonus'], dtype='object')

In [123]:
result.columns.get_level_values(1)

Index(['max', 'min', 'mean', 'sum', 'mean'], dtype='object')

In [124]:
[a+'_'+b for a, b in zip(result.columns.get_level_values(0), result.columns.get_level_values(1))]

['Salary_max', 'Salary_min', 'Salary_mean', 'Bonus_sum', 'Bonus_mean']

In [108]:
result['Salary']

,max,min,mean
Department,,,
Finance,100000,90000,95000.0
HR,60000,50000,55000.0
IT,80000,70000,75000.0


In [109]:
result.columns

MultiIndex([('Salary',  'max'),
            ('Salary',  'min'),
            ('Salary', 'mean'),
            ( 'Bonus',  'sum'),
            ( 'Bonus', 'mean')],
           )

In [110]:
result[('Salary',  'max')]

Department
Finance    100000
HR          60000
IT          80000
Name: (Salary, max), dtype: int64

In [112]:
for i in result.columns:
    print(i[0] + '_' + i[1])

Salary_max
Salary_min
Salary_mean
Bonus_sum
Bonus_mean


In [115]:
['_'.join(col) for col in result.columns]

['Salary_max', 'Salary_min', 'Salary_mean', 'Bonus_sum', 'Bonus_mean']

In [116]:
result.columns = ['_'.join(col) for col in result.columns]

In [118]:
result.reset_index()

,Department,Salary_max,Salary_min,Salary_mean,Bonus_sum,Bonus_mean
0,Finance,100000,90000,95000.0,19000,9500.0
1,HR,60000,50000,55000.0,11000,5500.0
2,IT,80000,70000,75000.0,15000,7500.0


In [119]:
result.columns

Index(['Salary_max', 'Salary_min', 'Salary_mean', 'Bonus_sum', 'Bonus_mean'], dtype='object')

**`apply`**

In [129]:
df['Salary'] * 2

0    100000
1    120000
2    140000
3    160000
4    180000
5    200000
Name: Salary, dtype: int64

In [136]:
def demo(x):
    print(type(x))
    return x * 2

df['Salary'].apply(demo)

<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>


0    100000
1    120000
2    140000
3    160000
4    180000
5    200000
Name: Salary, dtype: int64

In [137]:
df.apply(demo)

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


,Department,Employee,Salary,Bonus
0,HRHR,AliceAlice,100000,10000
1,HRHR,BobBob,120000,12000
2,ITIT,CharlieCharlie,140000,14000
3,ITIT,DavidDavid,160000,16000
4,FinanceFinance,EvaEva,180000,18000
5,FinanceFinance,FrankFrank,200000,20000


In [142]:
def debug(x):
    display(x)
    print('-'*50)
    return x

df.apply(debug, axis=1)

Department       HR
Employee      Alice
Salary        50000
Bonus          5000
Name: 0, dtype: object

--------------------------------------------------


Department       HR
Employee        Bob
Salary        60000
Bonus          6000
Name: 1, dtype: object

--------------------------------------------------


Department         IT
Employee      Charlie
Salary          70000
Bonus            7000
Name: 2, dtype: object

...

Department    Finance
Employee          Eva
Salary          90000
Bonus            9000
Name: 4, dtype: object

--------------------------------------------------


Department    Finance
Employee        Frank
Salary         100000
Bonus           10000
Name: 5, dtype: object

--------------------------------------------------


,Department,Employee,Salary,Bonus
0,HR,Alice,50000,5000
1,HR,Bob,60000,6000
2,IT,Charlie,70000,7000
3,IT,David,80000,8000
4,Finance,Eva,90000,9000
5,Finance,Frank,100000,10000


In [146]:
df['NameWithSalary'] = df.apply(lambda x: x['Employee'] + '-' + str(x['Salary']), axis=1)

In [147]:
df

,Department,Employee,Salary,Bonus,NameWithSalary
0,HR,Alice,50000,5000,Alice-50000
1,HR,Bob,60000,6000,Bob-60000
2,IT,Charlie,70000,7000,Charlie-70000
3,IT,David,80000,8000,David-80000
4,Finance,Eva,90000,9000,Eva-90000
5,Finance,Frank,100000,10000,Frank-100000


In [150]:
df['Department'].iloc[:2]

0    HR
1    HR
Name: Department, dtype: object

In [151]:
df.apply(lambda x: x.iloc[:2])

,Department,Employee,Salary,Bonus,NameWithSalary
0,HR,Alice,50000,5000,Alice-50000
1,HR,Bob,60000,6000,Bob-60000


In [152]:
df.iloc[:2]

,Department,Employee,Salary,Bonus,NameWithSalary
0,HR,Alice,50000,5000,Alice-50000
1,HR,Bob,60000,6000,Bob-60000


In [154]:
df.loc[:3]

,Department,Employee,Salary,Bonus,NameWithSalary
0,HR,Alice,50000,5000,Alice-50000
1,HR,Bob,60000,6000,Bob-60000
2,IT,Charlie,70000,7000,Charlie-70000
3,IT,David,80000,8000,David-80000


In [156]:
df_new = df.set_index('Employee')

In [159]:
df_new

,Department,Salary,Bonus,NameWithSalary
Employee,,,,
Alice,HR,50000,5000,Alice-50000
Bob,HR,60000,6000,Bob-60000
Charlie,IT,70000,7000,Charlie-70000
David,IT,80000,8000,David-80000
Eva,Finance,90000,9000,Eva-90000
Frank,Finance,100000,10000,Frank-100000


In [160]:
df_new.loc['Alice']

Department                 HR
Salary                  50000
Bonus                    5000
NameWithSalary    Alice-50000
Name: Alice, dtype: object

In [161]:
df_new.iloc[0]

Department                 HR
Salary                  50000
Bonus                    5000
NameWithSalary    Alice-50000
Name: Alice, dtype: object

In [162]:
df_new.iloc['Alice']

TypeError: Cannot index by location index with a non-integer key

In [165]:
df

,Department,Employee,Salary,Bonus,NameWithSalary
0,HR,Alice,50000,5000,Alice-50000
1,HR,Bob,60000,6000,Bob-60000
2,IT,Charlie,70000,7000,Charlie-70000
3,IT,David,80000,8000,David-80000
4,Finance,Eva,90000,9000,Eva-90000
5,Finance,Frank,100000,10000,Frank-100000


In [167]:
df.drop('NameWithSalary', axis=1, inplace=True)

In [168]:
df

,Department,Employee,Salary,Bonus
0,HR,Alice,50000,5000
1,HR,Bob,60000,6000
2,IT,Charlie,70000,7000
3,IT,David,80000,8000
4,Finance,Eva,90000,9000
5,Finance,Frank,100000,10000


In [173]:
def debug(x):
    print(type(x))
    print(x)
    print('-'*50)
    return x
df.map(debug)

<class 'str'>
HR
--------------------------------------------------
<class 'str'>
HR
--------------------------------------------------
<class 'str'>
IT
--------------------------------------------------
<class 'str'>
IT
--------------------------------------------------
<class 'str'>
Finance
--------------------------------------------------
<class 'str'>
Finance
--------------------------------------------------
<class 'str'>
Alice
--------------------------------------------------
<class 'str'>
Bob
--------------------------------------------------
<class 'str'>
Charlie
--------------------------------------------------
<class 'str'>
David
--------------------------------------------------
<class 'str'>
Eva
--------------------------------------------------
<class 'str'>
Frank
--------------------------------------------------
<class 'int'>
50000
--------------------------------------------------
<class 'int'>
60000
--------------------------------------------------
<class 'int'>
70

,Department,Employee,Salary,Bonus
0,HR,Alice,50000,5000
1,HR,Bob,60000,6000
2,IT,Charlie,70000,7000
3,IT,David,80000,8000
4,Finance,Eva,90000,9000
5,Finance,Frank,100000,10000


In [174]:
df.apply(debug)

<class 'pandas.core.series.Series'>
0         HR
1         HR
2         IT
3         IT
4    Finance
5    Finance
Name: Department, dtype: object
--------------------------------------------------
<class 'pandas.core.series.Series'>
0      Alice
1        Bob
2    Charlie
3      David
4        Eva
5      Frank
Name: Employee, dtype: object
--------------------------------------------------
<class 'pandas.core.series.Series'>
0     50000
1     60000
2     70000
3     80000
4     90000
5    100000
Name: Salary, dtype: int64
--------------------------------------------------
<class 'pandas.core.series.Series'>
0     5000
1     6000
2     7000
3     8000
4     9000
5    10000
Name: Bonus, dtype: int64
--------------------------------------------------


,Department,Employee,Salary,Bonus
0,HR,Alice,50000,5000
1,HR,Bob,60000,6000
2,IT,Charlie,70000,7000
3,IT,David,80000,8000
4,Finance,Eva,90000,9000
5,Finance,Frank,100000,10000


**`pipe`**

In [177]:
def add_column(df, col_name: str, value: int):
    df[col_name] = value
    return df

df.pipe(add_column, 'my_new_col', 10)

,Department,Employee,Salary,Bonus,new_col,my_new_col
0,HR,Alice,50000,5000,1,10
1,HR,Bob,60000,6000,1,10
2,IT,Charlie,70000,7000,1,10
3,IT,David,80000,8000,1,10
4,Finance,Eva,90000,9000,1,10
5,Finance,Frank,100000,10000,1,10


**Some analysing**

In [178]:
df_sales = pd.read_csv('data/adv/fact_internet_sales.csv')
df_customer = pd.read_csv('data/adv/dim_customer.csv')
df_currency = pd.read_csv('data/adv/dim_currency.csv')
df_product = pd.read_csv('data/adv/dim_product.csv')

In [188]:
df_currency.head()

,CurrencyKey,CurrencyAlternateKey,CurrencyName
0,1,AFA,Afghani
1,2,DZD,Algerian Dinar
2,3,ARS,Argentine Peso
3,4,AMD,Armenian Dram
4,5,AWG,Aruban Guilder


In [191]:
result = (
df_sales
    .merge(df_product[['ProductKey', 'EnglishProductName']], on='ProductKey')
    .drop(columns=['ProductKey'])
    .merge(
        df_customer[['CustomerKey','FirstName','MiddleName','LastName']],
        on='CustomerKey'
    )
    .drop(columns=['CustomerKey'])
    .merge(
        df_currency.drop('CurrencyAlternateKey', axis=1),
        on='CurrencyKey'
    )
    .drop(columns=['CurrencyKey'])
)


In [192]:
result.head()

,OrderDateKey,OrderQuantity,UnitPrice,ExtendedAmount,UnitPriceDiscountPct,DiscountAmount,ProductStandardCost,TotalProductCost,SalesAmount,TaxAmt,Freight,EnglishProductName,FirstName,MiddleName,LastName,CurrencyName
0,20101229,1,3578.2700,3578.2700,0.0,0.0,2171.2942,2171.2942,3578.2700,286.2616,89.4568,"Road-150 Red, 62",Cole,A,Watson,Canadian Dollar
1,20101229,1,3399.9900,3399.9900,0.0,0.0,1912.1544,1912.1544,3399.9900,271.9992,84.9998,"Mountain-100 Silver, 44",Rachael,M,Martinez,French Franc
2,20101229,1,3399.9900,3399.9900,0.0,0.0,1912.1544,1912.1544,3399.9900,271.9992,84.9998,"Mountain-100 Silver, 44",Sydney,S,Wright,US Dollar
3,20101229,1,699.0982,699.0982,0.0,0.0,413.1463,413.1463,699.0982,55.9279,17.4775,"Road-650 Black, 62",Ruben,NaN,Prasad,US Dollar
4,20101229,1,3399.9900,3399.9900,0.0,0.0,1912.1544,1912.1544,3399.9900,271.9992,84.9998,"Mountain-100 Silver, 44",Christy,NaN,Zhu,Australian Dollar


In [194]:
import sqlite3

con = sqlite3.connect('data/population/population.db')
df = pd.read_sql('select id, salary from population', con=con)
df.head()

,id,salary
0,1,368693
1,2,154398
2,3,1230304
3,4,1567795
4,5,1136950


In [200]:
df[df['salary'] < 200_000]['salary'].count() / df.shape[0]

np.float64(0.0987209202506653)

In [201]:
df.shape[0]

11649

In [202]:
len(df)

11649

In [203]:
df.head()

,id,salary
0,1,368693
1,2,154398
2,3,1230304
3,4,1567795
4,5,1136950


In [210]:
df_cat = pd.read_excel('data/population/population salary analysis.xlsx', usecols=['Salary Band'])
categories = df_cat.values.ravel().tolist()

In [211]:
categories

['till $200,000',
 '$200,001 - $400,000',
 '$400,001 - $600,000',
 '$600,001 - $800,000',
 '$800,001 - $1,000,000',
 '$1,000,001 - $1,200,000',
 '$1,200,001 - $1,400,000',
 '$1,400,001 - $1,600,000',
 '$1,600,001 - $1,800,000',
 '$1,800,001 and over']

In [213]:
bins = [0, 200_000, 400_000, 600_000, 800_000, 1_000_000, 1_200_000, 1_400_000, 1_600_000, 1_800_000, np.inf]

In [219]:
df['salary_cat'] = pd.cut(df['salary'], bins=bins, labels=categories)

In [220]:
df

,id,salary,salary_cat
0,1,368693,"$200,001 - $400,000"
1,2,154398,"till $200,000"
2,3,1230304,"$1,200,001 - $1,400,000"
3,4,1567795,"$1,400,001 - $1,600,000"
4,5,1136950,"$1,000,001 - $1,200,000"
...,...,...,...
11644,11645,1909496,"$1,800,001 and over"
11645,11646,830390,"$800,001 - $1,000,000"
11646,11647,24363,"till $200,000"
11647,11648,1869988,"$1,800,001 and over"


In [248]:
res = (
    df
    .groupby('salary_cat', observed=False)['salary']
    .agg(['mean', 'median', 'count'])
    .pipe(lambda df: df.assign(perc=df['count'] / df.shape[0]))
    .reset_index()
)

In [250]:
res[['perc', 'salary_cat', 'mean', 'median', 'count']]

,perc,salary_cat,mean,median,count
0,115.0,"till $200,000",9.922776e+04,98795.5,1150
1,117.0,"$200,001 - $400,000",2.995581e+05,299882.0,1170
2,123.4,"$400,001 - $600,000",4.991640e+05,497925.5,1234
3,115.6,"$600,001 - $800,000",6.996809e+05,701317.0,1156
4,117.5,"$800,001 - $1,000,000",9.011523e+05,899845.0,1175
5,122.6,"$1,000,001 - $1,200,000",1.098537e+06,1097838.0,1226
6,113.1,"$1,200,001 - $1,400,000",1.300685e+06,1300430.0,1131
7,113.2,"$1,400,001 - $1,600,000",1.499606e+06,1500623.0,1132
8,112.0,"$1,600,001 - $1,800,000",1.698519e+06,1697481.5,1120
9,115.5,"$1,800,001 and over",1.902892e+06,1906451.0,1155


In [226]:
df1 = df.copy()
df1

,id,salary,salary_cat
0,1,368693,"$200,001 - $400,000"
1,2,154398,"till $200,000"
2,3,1230304,"$1,200,001 - $1,400,000"
3,4,1567795,"$1,400,001 - $1,600,000"
4,5,1136950,"$1,000,001 - $1,200,000"
...,...,...,...
11644,11645,1909496,"$1,800,001 and over"
11645,11646,830390,"$800,001 - $1,000,000"
11646,11647,24363,"till $200,000"
11647,11648,1869988,"$1,800,001 and over"


In [229]:
df1['new_col'] = 1
df1 = df1.assign(another_col = 2)
df1

,id,salary,salary_cat,new_col,another_col
0,1,368693,"$200,001 - $400,000",1,2
1,2,154398,"till $200,000",1,2
2,3,1230304,"$1,200,001 - $1,400,000",1,2
3,4,1567795,"$1,400,001 - $1,600,000",1,2
4,5,1136950,"$1,000,001 - $1,200,000",1,2
...,...,...,...,...,...
11644,11645,1909496,"$1,800,001 and over",1,2
11645,11646,830390,"$800,001 - $1,000,000",1,2
11646,11647,24363,"till $200,000",1,2
11647,11648,1869988,"$1,800,001 and over",1,2


In [232]:
df2 = df['id'].iloc[:5]

In [242]:
def printt(x):
    print(x**2)
    return x

df2.apply(lambda x: (print(x**2), x)[1])

1
4
9
16
25


0    1
1    2
2    3
3    4
4    5
Name: id, dtype: int64

In [244]:
t = (print(1**2), 1)

1


In [245]:
t

(None, 1)

In [246]:
t[1]

1

In [247]:
(print(1**2), 1)[1]

1


1